In [1]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing


def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

model = EfficientDet()
print_model_summary(model)
load_weights_from_epoch = Config.load_weights_from_epoch
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))

Model: "efficient_det"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net (EfficientNet) multiple                  6771296   
_________________________________________________________________
bi_fpn (BiFPN)               multiple                  129126    
_________________________________________________________________
box_class_predict (BoxClassP multiple                  249069    
Total params: 7,149,491
Trainable params: 7,106,515
Non-trainable params: 42,976
_________________________________________________________________


In [2]:
dataset_list = tf.data.Dataset.list_files('./data/datasets/JPEGImages'+'\\*')

def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('./data/datasets/JPEGImages'+'\\*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (512,512))
    # image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
# Model has only one input so each data point has one element
    yield [image]
    

    #### 수정좀 해보기
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('./data/datasets/JPEGImages'+'\\*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (512,512))
    # image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
# Model has only one input so each data point has one element
    yield [image]

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: ./data/datasets/JPEGImages\\*'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model_quant = converter.convert()

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

In [ ]:
with open('./tflite_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [ ]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

In [ ]:
model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    return interpreter

def data_pre(data):
    
    return

def prediction(model, data):
    interpreter = model
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()

    input_data = np.array(data, dtype = np.uint8)

    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()
    output_data_class= interpreter.get_tensor(output_details[0]['index'])
    output_data_bb= interpreter.get_tensor(output_details[1]['index'])

    return output_data_class, output_data_bb

In [ ]:
dataset_list = tf.data.Dataset.list_files('./data/datasets/JPEGImages'+'\\*')
image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
# image = tf.cast(image / 255., tf.float32)
image = tf.expand_dims(image, 0)
print(image.shape)

In [ ]:
import numpy as np

model = load_model(model_path)

classification, regression = prediction(model, image)

print(classification.shape)
print(regression[0][1])

classification =  np.array(classification, dtype=np.float32)
regression =  np.array(regression, dtype=np.float32)